In [8]:
# 베이스 라인 모델로 Light BGM 모델을 사용할거임 
# 책에서는 파이썬 래퍼 LightGMB 과 사이킷런 LightGBM 중 파이썬 래퍼 LightBGM 사용

import os.path
from os import path
import pandas as pd

# 캐글 ↓제출경로
data_path = './input/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test  = pd.read_csv(data_path + 'test.csv' , index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

In [9]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1) # 타깃값 제거 

In [10]:
all_features = all_data.columns
all_features

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtype='obj

In [11]:
# 명목형 피처는 보통 웟-핫 인코딩 적용하는게 정석인듯 하다 

from sklearn.preprocessing import OneHotEncoder

# 명목형 피처 추출
cat_features = [feature for feature in all_features if 'cat' in feature]

onehot_encoder = OneHotEncoder()
#인코딩
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

encoded_cat_matrix

<1488028x184 sparse matrix of type '<class 'numpy.float64'>'
	with 20832392 stored elements in Compressed Sparse Row format>

In [12]:
#  필요없는 피처 제거 
# 1. calc 분류 속하는 20개 피처
# 2. ps_ind_10_bin, ps_ind_11_bin, ps_ind_12_bin, ps_ind_13_bin,
# 3. ps_ind_14,
# 4. ps_car_14

# 제거이유 

# 1 => 고윳값 별 타깃값 비율 차이가 없음. 타깃값 비율이 다르더라도 신뢰구간이 넓어 통계적 유효성이 떨어짐
# 2 => 신뢰구간이 넓어 통계적 유효성이 떨어짐 
# 3 => 타깃값 비율의 신뢰구간이 넓어 통계적 유효성이 떨어짐 
# 4 => ps_car_12 와 ps_car_14 두개가 강한 상관관계를 가지는데 이중 하나를 제거하는것이 ㄱㅊ다고 함 

In [13]:
# 추가로 제거할 피처 

drop_features = [
    'ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14'
]

# '1) 명목형 피처, 2) calc 분류의 피처 3) 추가 제거할 피처를 제외한 피처 
# 제외하고 남김 

# 아 명목형 피처는 원핫 인코딩에 사용할 예정이므로 cat 은 제거한다 
# 괜히 헷갈렸군.........
remaining_features = [
    feature for feature in all_features
    if ( 'cat' not in feature and
        'calc' not in feature and
        feature not in drop_features
        )
]

remaining_features

['ps_ind_01',
 'ps_ind_03',
 'ps_ind_06_bin',
 'ps_ind_07_bin',
 'ps_ind_08_bin',
 'ps_ind_09_bin',
 'ps_ind_15',
 'ps_ind_16_bin',
 'ps_ind_17_bin',
 'ps_ind_18_bin',
 'ps_reg_01',
 'ps_reg_02',
 'ps_reg_03',
 'ps_car_11',
 'ps_car_12',
 'ps_car_13',
 'ps_car_15']

In [14]:
from scipy import sparse 

# remaining_features 와 앞서 명목형 피처들 원핫 인코딩한 encoded_cat_matrix 를 합쳐줌 

all_data_sprs =sparse.hstack([sparse.csr_matrix(all_data[remaining_features]),
    encoded_cat_matrix],
    format ='csr'
)

# csr_matrix() 는 전달받은 데이터는 CSR 형식으로 바꿔줌 
# hstack 은 행렬을 수평방향으로 합침 

In [15]:
# 데이터 나누기

num_train = len(train)

# 훈련 데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values 
print(y)

[0 0 0 ... 0 0 0]


In [30]:
# 평가지표 계산 함수 작성 

import numpy as np

def eval_gini(y_true, y_pred):
    # 실젯값과 예측값의 크기가 서로 같은지 확인 (값이 다르면 오류 발생 )
    
    assert y_true.shape == y_pred.shape
    
    n_samples = y_true.shape[0] # 데이터 개수
    L_mid = np.linspace( 1/ n_samples, 1, n_samples) # 1/n_samples~ 1 까지 n_samples-1 개의 구간으로 나누겠다. 
    
    # 1) 예측값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()]
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선
    G_pred = np.sum(L_mid - L_pred)
    
    # 2) 예측 완벽할때 지니계수
    true_order = y_true[y_true.argsort()]
    L_true = np.cumsum(true_order) / np.sum(true_order)
    G_true = np.sum(L_mid - L_true)
    
    # 정규화된 지니계수
    return G_pred / G_true
        

In [31]:
# LightGMB 용 gini() 함수 

def gini(preds, dtrain):
    labels = dtrain.get_label() # 
    return 'gini', eval_gini(labels, preds), True # 반환값 
 
# 반환값 리스트
# 평가지표 이름, 평가점수 , 평가 점수가 높을수록 좋은지 여부 

In [32]:
# OOF 방식으로 Light GBM 훈련 

from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기
# 시계열 데이터가 아니라면 섞어도 되는데 시계열 데이터는 절대로 섞으면 안된다. 
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

params = {
    'objective' : 'binary',
    'learning_rate' : 0.01,
    'force_row_wise': True,
    'random_state': 0
}

In [33]:
# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열 
oof_val_preds = np.zeros(X.shape[0])

# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0])

In [34]:
import lightgbm as lgb

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터
    
    # LightGMB 전용 데이터셋 생성
    dtrain = lgb.Dataset(X_train, y_train)
    dvalid = lgb.Dataset(X_valid, y_valid)
    
    # LightGBM 전용 데이터셋 생성
    dtrain = lgb.Dataset(X_train, y_train) # LightGBM 전용 훈련 데이터셋
    dvalid = lgb.Dataset(X_valid, y_valid) # LightGMB 전용 검증 데이터셋
    
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=params,    # 훈련용 하이퍼 파라미터 
                          train_set=dtrain, # 훈련 데이터셋
                          num_boost_round=1000, # 부스팅 반복 횟수
                          valid_sets=dvalid, # 성능 평가용 검증 데이터셋
                          feval=gini, # 검증용 평가지표
                          early_stopping_rounds=100, # 조기종료 조건
                          verbose_eval=100 # 100번쨰마다 점수 출력
                          )
    
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)
    
    # 검증 데이터 예측 확률에 대한 정규화 지니계수 
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수 : {gini_score} \n')
    
    
    

######################################## 폴드 1 / 폴드 5 ########################################


c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.153354	valid_0's gini: 0.261651
[200]	valid_0's binary_logloss: 0.152426	valid_0's gini: 0.275704
[300]	valid_0's binary_logloss: 0.152023	valid_0's gini: 0.282277
[400]	valid_0's binary_logloss: 0.1518	valid_0's gini: 0.286648
[500]	valid_0's binary_logloss: 0.151713	valid_0's gini: 0.287944
[600]	valid_0's binary_logloss: 0.151672	valid_0's gini: 0.288637
[700]	valid_0's binary_logloss: 0.151659	valid_0's gini: 0.288939
Early stopping, best iteration is:
[681]	valid_0's binary_logloss: 0.151659	valid_0's gini: 0.289034
폴드 1 지니계수 : 0.2890336154188232 

#########

In [35]:
print('OOF 검증 데이터 ', eval_gini(y, oof_val_preds))

OOF 검증 데이터  0.2804995714877777


In [36]:
submission['target'] = oof_test_preds
submission.to_csv('./output/submission.csv')